<a href="https://colab.research.google.com/github/Damilola-ops/MultiLingual-Named-Entity-Recognition-/blob/main/MultiLingualNamedEntityRecognitionipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install huggingface-hub
pip install datasets
pip install transformers
pip install seqeval


In [ ]:
from datasets import get_dataset_config_names

In [ ]:
from collections import defaultdict
from datasets import DatasetDict

In [ ]:
xtreme_subsets = get_dataset_config_names('xtreme')
print(f'XTREME has {len(xtreme_subsets)} configurations')
x_subsets = [s for s in xtreme_subsets if s.startswith('PAN')]
panx_subsets[:3]

In [ ]:
languages = ['de','fr','it','en']
fracs = [0.629,0.229,0.084,0.059]
panx_ch = defaultdict(DatasetDict)

for lang,frac in zip(languages,fracs):
    ds = load_dataset('xtreme',name=f'PAN-X.{lang}')
    for split in ds:
        panx_ch[lang][split] = (
            ds[split].shuffle(seed=0).select(range(int(frac*ds[split].num_rows)))
            
            
    
        )

In [ ]:
pd.DataFrame({lang:[panx_ch[lang]['train'].num_rows] for lang in languages},
             index=['Number of training examples'])

In [ ]:
mport torch.nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

In [ ]:
from transformers import AutoTokenizer 
bert = 'bert-base-uncased'
xlmr_model_name = 'xlm-roberta-base'
bert_tokenizer = AutoTokenizer.from_pretrained(bert)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

In [ ]:
    
    def __init__(self,config):
        super().__init__(config)
        self.num_labels = config.num_labels
        #model body 
        self.roberta = RobertaModel(config,add_pooling_layer=False)
        #token classification head
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size,config.num_labels)
        self.init_weights()
    
    def forward(self,input_ids=None,attention_mask=None,token_type_ids=None,labels=None,**kwargs):
        #use model body to get encoder representations 
        outputs = self.roberta(input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,**kwargs)
        #apply classifier to encode representation 
        sequence_outputs = self.dropout(outputs[0])
        logits = self.classifier(sequence_outputs)
        #calculate losses 
        loss = None
        if labels is not None :
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1,self.num_labels),labels.view(-1))
        return TokenClassifierOutput(loss=loss,logits=logits,hidden_states=outputs.hidden_states,
                                    attentions=outputs.attentions)
    
        